1. Загружаем сэмплы
2. Новые точки
3. Считаем для них MC оценки


In [1]:
# %load_ext autoreload
# %autoreload 2

In [2]:
from matplotlib import pyplot as plt
%matplotlib inline

In [3]:
import torch

In [4]:
from tqdm import tqdm

In [5]:
from easydict import EasyDict as edict
args = {
    #'bnn_lr': 1e-3,
    'cv_lr': 1e-7,
    'n_cv_iter': 100,
    'batch_size' : 20000,
    'input_dim' : 784,
    'width' : 100,
    'depth' : 2,
    'output_dim' : 2,
    #'n_epoch' : 200,
    #'alpha0' : 10, 
    #'beta0' : 10,
    #'resample_prior_every' : 15,
    #'resample_momentum_every' : 50,
    #'burn_in_epochs' : 20,
    #'save_freq' : 4,
    #'resample_prior_until' : 100,
    #'report_every' : 10,
    }

args = edict(args)

In [6]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [7]:
import pickle
from copy import deepcopy
with open('../saved_samples/mnist_weights/30_samples_seed-1.pkl', 'rb') as fp:
    samples = pickle.load(fp)

#samples = samples[-10:]

In [8]:
import sys
sys.path.append('..')

In [9]:
from control_variates.model import LogRegression

In [10]:
from mnist_utils import load_mnist_dataset

In [11]:
train_dl, valid_dl = load_mnist_dataset('../data/mnist/', args.batch_size, classes=[3, 5])

In [12]:
N_train = len(train_dl.dataset)

In [13]:
len(train_dl)

1

In [14]:
#samples = [(x[0][-10:], x[1]) for x in samples]

In [15]:
trajectories = [[LogRegression(args.input_dim)
                 for j in range(len(samples[i][0]))]
                for i in range(len(samples))]

for i in range(len(samples)):
    for j in range(len(samples[i][0])):
        trajectories[i][j].load_state_dict(samples[i][0][j])

priors = [samples[i][1] for i in range(len(samples))]

In [16]:
trajectories[0][1]

LogRegression(
  (linear): Linear(in_features=784, out_features=2, bias=True)
)

In [17]:
x_new, y_new = next(iter(valid_dl))
train_x, train_y = next(iter(train_dl))

Фитим ncv

In [18]:
from control_variates.cv import PsyLinear, SteinCV, PsyConstVector, PsyMLP
from control_variates.cv_utils import state_dict_to_vec, compute_naive_variance
from control_variates.model import get_binary_prediction
import numpy as np

In [19]:
from control_variates.uncertainty_quantification import ClassificationUncertaintyMCMC

In [20]:
psy_input_dim = 1570

In [21]:
#tqdm._instances.clear()

In [22]:
def centr_regularizer(ncv, models, x, ll_div=None):
    return (ncv(models, x, ll_div).mean(0))**2

In [23]:
from control_variates.cv_utils import compute_log_likelihood, compute_concat_gradient

def compute_ll_div(models, train_x, train_y, N_train, priors=None):
    for model in models:
        model.zero_grad()
    log_likelihoods = [(compute_log_likelihood(train_x, train_y, model) * N_train).backward() for model in models]
    ll_div = torch.stack([compute_concat_gradient(model, priors) for model in models])
    return ll_div


In [24]:
ncv_s = []
squeezed_weights = [[state_dict_to_vec(samples[i][0][j]) 
                     for j in range(len(samples[i][0]))]
                    for i in range(len(samples))]
psy_input_dim = squeezed_weights[0][0].shape[0]

for models, pr in zip(trajectories, priors):
    #psy_model = PsyConv(psy_input_dim, 75)
    psy_model = PsyConstVector(psy_input_dim)
    #psy_model = PsyLinear(psy_input_dim)
    #psy_model = PsyMLP(psy_input_dim, 50, 1)
    psy_model.init_zero()
    psy_model.to(device)

    neural_control_variate = SteinCV(psy_model, train_x, train_y, pr, N_train)

    ncv_optimizer = torch.optim.Adam(psy_model.parameters(), lr=args.cv_lr, weight_decay=0.0) #1e-4)
    uncertainty_quant = ClassificationUncertaintyMCMC(models, neural_control_variate)

    train_x, train_y = next(iter(train_dl))
    ll_div = compute_ll_div(models, train_x, train_y, N_train, priors=priors)

    function_f = lambda model, x: get_binary_prediction(model, x, classes=[0, 1])
    history = [] 
    x = x_new[13:14]
    #fig = plt.figure()

    
    for it in tqdm(range(args.n_cv_iter)):
        #try:
        #    train_x, train_y = next(data_iter)
        #except:
        #    data_iter = iter(train_dl)
        #    train_x, train_y = next(data_iter)
        #neural_control_variate.train_x = train_x
        #neural_control_variate.train_y = train_y
        ncv_optimizer.zero_grad()
        mc_variance, no_cv_variance = compute_naive_variance(function_f, neural_control_variate, models, x, ll_div)
        history.append(mc_variance.mean().item())
        (mc_variance + 1e-3*centr_regularizer(neural_control_variate, models, x, ll_div)).mean().backward()
        ncv_optimizer.step()
    ncv_s.append(neural_control_variate)
    print(mc_variance.mean().item(), no_cv_variance.mean().item())
    # plt.plot(np.arange(it+1), history)
    # plt.axhline(y=no_cv_variance.mean(), color='r', linestyle='-')
    # plt.xlim(0, args.n_cv_iter)
    # plt.ylim(0, 0.02)
    # plt.show()
    print(neural_control_variate(models, x).mean())

 19%|█▉        | 19/100 [00:20<01:40,  1.24s/it]hi


KeyboardInterrupt: 

In [25]:
psy_weights = [deepcopy(ncv.psy_model.state_dict()) for ncv in ncv_s]

In [26]:
psy_weights[0], priors[0], trajectories[0][0].state_dict()

IndexError: list index out of range

In [27]:
cv = SteinCV(psy_model, train_x, train_y, priors[0], N_train)
cv.psy_model.load_state_dict(psy_weights[0])

IndexError: list index out of range

In [28]:
cv(trajectories[0], x_new[13:14]).mean()

tensor(0., grad_fn=<MeanBackward0>)

In [29]:
ncv_s[0](trajectories[0], x_new[13:14]).mean()

IndexError: list index out of range

In [30]:
x_new[13:14].max()

tensor(2.8215)

In [31]:
with open('../data/psy_weights.pckl', 'wb') as fp:
    pickle.dump(psy_weights, fp)

In [32]:
import pickle
with open('../data/psy_weights.pckl', 'rb') as fp:
    psy_weights = pickle.load(fp)

In [33]:
from control_variates.cv_utils import trapezoidal_kernel, SpectralVariance

In [34]:
models

[LogRegression(
   (linear): Linear(in_features=784, out_features=2, bias=True)
 ), LogRegression(
   (linear): Linear(in_features=784, out_features=2, bias=True)
 ), LogRegression(
   (linear): Linear(in_features=784, out_features=2, bias=True)
 ), LogRegression(
   (linear): Linear(in_features=784, out_features=2, bias=True)
 ), LogRegression(
   (linear): Linear(in_features=784, out_features=2, bias=True)
 ), LogRegression(
   (linear): Linear(in_features=784, out_features=2, bias=True)
 ), LogRegression(
   (linear): Linear(in_features=784, out_features=2, bias=True)
 ), LogRegression(
   (linear): Linear(in_features=784, out_features=2, bias=True)
 ), LogRegression(
   (linear): Linear(in_features=784, out_features=2, bias=True)
 ), LogRegression(
   (linear): Linear(in_features=784, out_features=2, bias=True)
 ), LogRegression(
   (linear): Linear(in_features=784, out_features=2, bias=True)
 ), LogRegression(
   (linear): Linear(in_features=784, out_features=2, bias=True)
 ), Log

In [35]:
ncv_s2 = []
for models, pr in zip(trajectories, priors):
    psy_model = PsyConstVector(psy_input_dim)
    psy_model.init_zero()
    psy_model.to(device)

    neural_control_variate = SteinCV(psy_model, train_x, train_y, priors, N_train)
    function_f = lambda model, x: get_binary_prediction(model, x, classes=[0, 1])

    function_h = lambda model, x: function_f(model, x) - neural_control_variate(model, x)

    window_lag_f = trapezoidal_kernel
    truncation_point = 40 #len(models) 
    spectral_loss = SpectralVariance(function_h, models, window_lag_f, truncation_point)
    no_cv_loss = SpectralVariance(function_f, models, window_lag_f, truncation_point)

    ncv_optimizer = torch.optim.Adam(psy_model.parameters(), lr=args.cv_lr, weight_decay=0.0) #1e-4)

    history = [] 
    x = x_new[13:14]
    #fig = plt.figure()

    no_cv_variance = no_cv_loss(x)

    data_iter = iter(train_dl)
    for it in tqdm(range(args.n_cv_iter)):
        try:
            train_x, train_y = next(data_iter)
        except:
            data_iter = iter(train_dl)
            train_x, train_y = next(data_iter)
        neural_control_variate.train_x = train_x
        neural_control_variate.train_y = train_y
        ncv_optimizer.zero_grad()
        mc_variance = spectral_loss(x)
        history.append(mc_variance.mean().item())
        (mc_variance + 1e-4*centr_regularizer(neural_control_variate, models, x)).backward()
        ncv_optimizer.step()
    ncv_s2.append(neural_control_variate)
    print(mc_variance.mean().item(), no_cv_variance.mean().item())
    print(neural_control_variate(models, x).mean())
    # plt.plot(np.arange(it+1), history)
    # plt.axhline(y=no_cv_variance.mean(), color='r', linestyle='-')
    # plt.xlim(0, args.n_cv_iter)
    # plt.show()

  4%|▍         | 4/100 [00:19<07:37,  4.76s/it]

In [34]:
psy_weights2 = [deepcopy(ncv.psy_model.state_dict()) for ncv in ncv_s2]

In [35]:
with open('../data/psy_weights2.pckl', 'wb') as fp:
    pickle.dump(psy_weights2, fp)